# Import Packages and Load Data

In [1]:
!pip install nltk import numpy as np
 import numpy as np


In [7]:

!pip install --no-cache-dir --force-reinstall numpy==1.24.4 --upgrade --ignore-installed
print(np.__version__)  # should be 1.24.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 347.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
2.2.5


In [1]:
import pandas as pd
import boto3
import zipfile
import io
import os
import json
import ast
import pandas as pd

import numpy as np
#cleaning
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Unzipping Main Folder

In [3]:
bucket = 'msba-trends-group-trendnav'
zip_key = 'Dataset-20250422T012041Z-001.zip'

s3 = boto3.client('s3')
response = s3.get_object(Bucket=bucket, Key=zip_key)

# Extract directly from S3 stream
with zipfile.ZipFile(io.BytesIO(response['Body'].read())) as zip_ref:
    zip_ref.extractall('unzipped_s3_data')

# List files
print(os.listdir('unzipped_s3_data'))

['Dataset']


In [4]:
folder_list=[]
for root, dirs, files in os.walk('unzipped_s3_data'):
    for file in files:
        print(os.path.join(root, file))
        folder_list.append(os.path.join(root, file))

unzipped_s3_data/Dataset/Other Datasets.zip
unzipped_s3_data/Dataset/reddit_trending_products.json
unzipped_s3_data/Dataset/reddit_trending_products_v2.json
unzipped_s3_data/Dataset/retail_sales_dataset.csv
unzipped_s3_data/Dataset/Amazon-QuesAns.zip


In [5]:
import zipfile
zip_to_extract = 'unzipped_s3_data/Dataset/Amazon-QuesAns.zip'
with zipfile.ZipFile(zip_to_extract, 'r') as zip_ref:
    zip_ref.extractall('unzipped_s3_data/Dataset/Amazon-QuesAns')

# List contents
import os
print(os.listdir('unzipped_s3_data/Dataset/Amazon-QuesAns'))

['qa_Appliances.json', 'qa_Musical_Instruments.json', 'qa_Patio_Lawn_and_Garden.json', 'qa_Pet_Supplies.json', 'qa_Video_Games.json', 'qa_Home_and_Kitchen.json', 'qa_Grocery_and_Gourmet_Food.json', 'qa_Tools_and_Home_Improvement.json', 'qa_Arts_Crafts_and_Sewing.json', 'qa_Electronics.json', 'qa_Office_Products.json', 'qa_Software.json', 'qa_Beauty.json', 'qa_Baby.json', 'qa_Toys_and_Games.json', 'qa_Industrial_and_Scientific.json', 'qa_Clothing_Shoes_and_Jewelry.json', 'qa_Cell_Phones_and_Accessories.json', 'qa_Automotive.json', 'qa_Sports_and_Outdoors.json', 'qa_Health_and_Personal_Care.json', '__MACOSX']


### Load Data into Df

In [2]:

# Step 1: Load raw JSON
with open('unzipped_s3_data/Dataset/reddit_trending_products.json') as f:
    raw = json.load(f)

# Step 2: Convert each column (dict) to a list, then zip into rows
columns = list(raw.keys())
records = list(zip(*[list(raw[col].values()) for col in columns]))

# Step 3: Reconstruct DataFrame
reddit = pd.DataFrame(records, columns=columns)
reddit['created_at'] = pd.to_datetime(reddit['created_utc'], unit='ms')
reddit = reddit.drop(columns='created_utc')  # optional: drop the old column

reddit.head(2)


,subreddit,title,score,num_comments,url,flair,id,created_at
0,BuyItForLife,Me with the Golden Taco the day I bought it vs...,14466,448,https://i.redd.it/lsbf6wap5fte1.jpeg,Vintage,1jtlx8r,2025-04-07 14:00:09
1,BuyItForLife,"This 1980's marker, still better than any mode...",9823,235,https://i.redd.it/n65twfbq1hue1.jpeg,Vintage,1jxrtwd,2025-04-12 21:25:32


In [3]:
print(os.listdir('unzipped_s3_data/Dataset/Amazon-QuesAns'))
col_list = ['asin', 'question', 'answer', 'questionType', 'answerType', 'answerTime', 'unixTime','category']
amazon_qa = pd.DataFrame(columns=col_list)
print(amazon_qa)

def amazon_qa_fetch(filename_qa,col_list,cat):
    global amazon_qa
    records = []
    fail_count = 0
    with open(filename_qa, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            try:
                # Use ast.literal_eval for Python-like dict parsing
                parsed = ast.literal_eval(line.strip())
                records.append(parsed)
            except Exception as e:
                print(f" Failed on line {i}: {e}")
                fail_count += 1
    print(filename_qa)
    print(f"Parsed {len(records)} records")
    print(f"Skipped {fail_count} lines")

    amazon_qa_subgroup = pd.DataFrame(records)
    amazon_qa_subgroup['category'] = cat.replace('.json', '')
    amazon_qa_subgroup = amazon_qa_subgroup[col_list] 
    if set(amazon_qa.columns) == set(amazon_qa_subgroup.columns):
        amazon_qa = pd.concat([amazon_qa, amazon_qa_subgroup]).drop_duplicates().reset_index(drop=True)
    return amazon_qa

['qa_Appliances.json', 'qa_Musical_Instruments.json', 'qa_Patio_Lawn_and_Garden.json', 'qa_Pet_Supplies.json', 'qa_Video_Games.json', 'qa_Home_and_Kitchen.json', 'qa_Grocery_and_Gourmet_Food.json', 'qa_Tools_and_Home_Improvement.json', 'qa_Arts_Crafts_and_Sewing.json', 'qa_Electronics.json', 'qa_Office_Products.json', 'qa_Software.json', 'qa_Beauty.json', 'qa_Baby.json', 'qa_Toys_and_Games.json', 'qa_Industrial_and_Scientific.json', 'qa_Clothing_Shoes_and_Jewelry.json', 'qa_Cell_Phones_and_Accessories.json', 'qa_Automotive.json', 'qa_Sports_and_Outdoors.json', 'qa_Health_and_Personal_Care.json', '__MACOSX']
Empty DataFrame
Columns: [asin, question, answer, questionType, answerType, answerTime, unixTime, category]
Index: []


In [4]:
qa_file_list=os.listdir('unzipped_s3_data/Dataset/Amazon-QuesAns')
for cat in qa_file_list:
    if '.json' in cat:
        file='unzipped_s3_data/Dataset/Amazon-QuesAns/'+str(cat)
        print(cat)
        amazon_qa_fetch(file,col_list,cat)

qa_Appliances.json
unzipped_s3_data/Dataset/Amazon-QuesAns/qa_Appliances.json
Parsed 9011 records
Skipped 0 lines


/tmp/ipykernel_34938/1954438945.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  amazon_qa = pd.concat([amazon_qa, amazon_qa_subgroup]).drop_duplicates().reset_index(drop=True)


qa_Musical_Instruments.json
unzipped_s3_data/Dataset/Amazon-QuesAns/qa_Musical_Instruments.json
Parsed 23322 records
Skipped 0 lines
qa_Patio_Lawn_and_Garden.json
unzipped_s3_data/Dataset/Amazon-QuesAns/qa_Patio_Lawn_and_Garden.json
Parsed 59595 records
Skipped 0 lines
qa_Pet_Supplies.json
unzipped_s3_data/Dataset/Amazon-QuesAns/qa_Pet_Supplies.json
Parsed 36607 records
Skipped 0 lines
qa_Video_Games.json
unzipped_s3_data/Dataset/Amazon-QuesAns/qa_Video_Games.json
Parsed 13307 records
Skipped 0 lines
qa_Home_and_Kitchen.json
unzipped_s3_data/Dataset/Amazon-QuesAns/qa_Home_and_Kitchen.json
Parsed 184439 records
Skipped 0 lines
qa_Grocery_and_Gourmet_Food.json
unzipped_s3_data/Dataset/Amazon-QuesAns/qa_Grocery_and_Gourmet_Food.json
Parsed 19538 records
Skipped 0 lines
qa_Tools_and_Home_Improvement.json
unzipped_s3_data/Dataset/Amazon-QuesAns/qa_Tools_and_Home_Improvement.json
Parsed 101088 records
Skipped 0 lines
qa_Arts_Crafts_and_Sewing.json
unzipped_s3_data/Dataset/Amazon-QuesAns/qa_A

In [7]:
amazon_qa.head(3)

,asin,question,answer,questionType,answerType,answerTime,unixTime,category
0,B00004U9JP,I have a 9 year old Badger 1 that needs replac...,I replaced my old one with this without a hitch.,yes/no,?,"Jun 27, 2014",1.403852e+09,qa_Appliances
1,B00004U9JP,model number,This may help InSinkErator Model BADGER-1: Bad...,open-ended,NaN,"Apr 28, 2014",1.398668e+09,qa_Appliances
2,B00004U9JP,can I replace Badger 1 1/3 with a Badger 5 1/2...,Plumbing connections will vary with different ...,yes/no,?,"Aug 25, 2014",1.408950e+09,qa_Appliances


# Data Cleaning

In [78]:

amazon_qa['created_at'] = pd.to_datetime(amazon_qa['unixTime'], unit='s')
amazon_qa
amazon_review=amazon_qa.copy()

In [79]:
amazon_review['year'] = amazon_review['created_at'].dt.year
amazon_review.head(2)

,asin,question,answer,questionType,answerType,answerTime,unixTime,category,created_at,year
0,B00004U9JP,I have a 9 year old Badger 1 that needs replac...,I replaced my old one with this without a hitch.,yes/no,?,"Jun 27, 2014",1.403852e+09,qa_Appliances,2014-06-27 07:00:00,2014.0
1,B00004U9JP,model number,This may help InSinkErator Model BADGER-1: Bad...,open-ended,NaN,"Apr 28, 2014",1.398668e+09,qa_Appliances,2014-04-28 07:00:00,2014.0


In [80]:
amazon_review.groupby(['year'])['asin'].count()

year
2006.0      1257
2007.0      2809
2008.0      3697
2009.0      4696
2010.0      4873
2011.0      5195
2012.0     16493
2013.0    337987
2014.0    759305
2015.0    198038
Name: asin, dtype: int64

In [92]:
amazon_filt=amazon_review[amazon_review['year']>=2014.0]

In [93]:
amazon_filt.year.min()

2014.0

In [94]:
amazon_filt['category']=amazon_filt['category'].str.replace('qa_', '')
amazon_filt.category.unique()

/tmp/ipykernel_34938/2728456877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_filt['category']=amazon_filt['category'].str.replace('qa_', '')


array(['Appliances', 'Musical_Instruments', 'Patio_Lawn_and_Garden',
       'Pet_Supplies', 'Video_Games', 'Home_and_Kitchen',
       'Grocery_and_Gourmet_Food', 'Tools_and_Home_Improvement',
       'Arts_Crafts_and_Sewing', 'Electronics', 'Office_Products',
       'Software', 'Beauty', 'Baby', 'Toys_and_Games',
       'Industrial_and_Scientific', 'Clothing_Shoes_and_Jewelry',
       'Cell_Phones_and_Accessories', 'Automotive', 'Sports_and_Outdoors',
       'Health_and_Personal_Care'], dtype=object)

In [84]:
amazon_filt=amazon_filt[amazon_filt['category'].isin(['Home_and_Kitchen','Software','Cell_Phones_and_Accessories', 'Electronics','Health_and_Personal_Care',
      'Toys_and_Games', 'Clothing_Shoes_and_Jewelry', 'Beauty', 'Appliances', 'Video_Games'])].copy()

(73798, 10)

In [95]:
amazon_filt.to_csv('amazon_filt2.csv')
#amazon_filt=pd.read_csv('amazon_filt')

In [96]:
#amazon_filt.reset_index(inplace=True)

In [97]:
amazon_filt.groupby('category')['question'].count()

category
Appliances                       6592
Arts_Crafts_and_Sewing          15291
Automotive                      65396
Baby                            20105
Beauty                          30672
Cell_Phones_and_Accessories     54227
Clothing_Shoes_and_Jewelry      16743
Electronics                    201566
Grocery_and_Gourmet_Food        13833
Health_and_Personal_Care        57973
Home_and_Kitchen               123229
Industrial_and_Scientific        9063
Musical_Instruments             16894
Office_Products                 30387
Patio_Lawn_and_Garden           42585
Pet_Supplies                    26116
Software                         5155
Sports_and_Outdoors            105995
Tools_and_Home_Improvement      72805
Toys_and_Games                  33976
Video_Games                      8740
Name: question, dtype: int64

In [98]:
amazon_filt.year.min()

2014.0

In [99]:
reddit_all=reddit.copy()
reddit_all.flair.unique()

array(['Vintage', 'Currently sold', 'Review', 'Discussion', '[Request]',
       'Repair', 'Meta', '[Request] Answered!', 'BIFL Skills', 'Warranty',
       '👚Clothing & Shoes ', '🍎 Food ', '🚿 Personal Care ',
       '🏠 Home & Apartment ', '💬 Meta Discussion ',
       '♻️ Recycling & Zero-Waste ', '⛹️ Hobbies', '🚧 DIY & Repair ',
       '💰 Finance & Bills', '🚗 Auto ', '🎓\xa0Education / Philosophy',
       '🌱 Gardening ', '🧒 Children & Childcare ', '📱 Phone & Internet',
       '📦 Secondhand', '🏆 Buy It For Life ', '🧽 Cleaning & Organization',
       '👀 Glasses & Contacts', '✈️ Travel & Transport', '💻 Electronics ',
       None, '👟Fitness', 'cars & automotives', 'clothing & jewelry',
       'computers & electronics', 'META', 'home and garden',
       'food & drink', 'tools', 'Misc', 'Desktops / Laptops', 'Gaming',
       'Watches', 'Phones', 'Music', 'TV / Projectors',
       'Computer peripherals', 'Medical', 'VR / AR', 'Wearables',
       'Transportation', 'Drones / UAVs', 'Cameras', 'Pr

In [100]:
reddit_all['created_at'].min()

Timestamp('2025-03-21 03:34:04')

In [101]:
reddit_all['created_at'].max()

Timestamp('2025-04-20 02:30:32')

In [102]:
reddit_all=reddit_all[~reddit_all['flair'].isin(["[Request]",'Miscellaneous','Rumour','News/Rumour','Meme/Macro','Space','Politics','Misc',
       'Networking/Telecom', 'Net Neutrality', 'Story','Screenshot','Tech Support Solved','News','BIFL Skills', 'Warranty'])]
reddit_all.flair.value_counts()

flair
Discussion           445
Build Help           305
Support              210
🍎 Food               138
Business             122
                    ... 
Music                  1
Review Megathread      1
Drones / UAVs          1
Machine Learning       1
Wearables              1
Name: count, Length: 89, dtype: int64

In [103]:
reddit_all['title_lowercase']=reddit_all['title'].str.lower()
reddit_all_backup=reddit_all.copy()
reddit_all = reddit_all.drop_duplicates(subset=['title_lowercase'], keep='last')

In [104]:
reddit_all['year'] = reddit_all['created_at'].dt.year

reddit_all.head(2)

,subreddit,title,score,num_comments,url,flair,id,created_at,title_lowercase,year
0,BuyItForLife,Me with the Golden Taco the day I bought it vs...,14466,448,https://i.redd.it/lsbf6wap5fte1.jpeg,Vintage,1jtlx8r,2025-04-07 14:00:09,me with the golden taco the day i bought it vs...,2025
1,BuyItForLife,"This 1980's marker, still better than any mode...",9823,235,https://i.redd.it/n65twfbq1hue1.jpeg,Vintage,1jxrtwd,2025-04-12 21:25:32,"this 1980's marker, still better than any mode...",2025


In [105]:
reddit_final=reddit_all[['subreddit','title','created_at','flair','year']]

In [106]:
amazon_filt[['question','answer','created_at','category','year']]

,question,answer,created_at,category,year
0,I have a 9 year old Badger 1 that needs replac...,I replaced my old one with this without a hitch.,2014-06-27 07:00:00,Appliances,2014.0
1,model number,This may help InSinkErator Model BADGER-1: Bad...,2014-04-28 07:00:00,Appliances,2014.0
2,can I replace Badger 1 1/3 with a Badger 5 1/2...,Plumbing connections will vary with different ...,2014-08-25 07:00:00,Appliances,2014.0
3,Does this come with power cord and dishwasher ...,It does not come with a power cord. It does co...,2014-11-03 08:00:00,Appliances,2014.0
4,loud noise inside when turned on. sounds like ...,Check if you dropped something inside.Usually ...,2014-06-21 07:00:00,Appliances,2014.0
...,...,...,...,...,...
1383858,Do the lights rotate by it self? Or do I have ...,"Hi Fine Tuning, Deneve here. Our aroma diffuse...",2014-07-21 07:00:00,Health_and_Personal_Care,2014.0
1383859,I see that it comes with an adapter but in the...,"Yes, you need to plug it on the electrical out...",2014-08-06 07:00:00,Health_and_Personal_Care,2014.0
1383860,does this diffuser automatically shut off when...,"Hi Pam, Deneve here. Yes, our diffuser will au...",2014-07-19 07:00:00,Health_and_Personal_Care,2014.0
1383861,Do you have to have the LED light on? Or can y...,"Hi Christina, Deneve here. Our essential oil d...",2014-07-28 07:00:00,Health_and_Personal_Care,2014.0


In [107]:
amazon_filt.head(2)

,asin,question,answer,questionType,answerType,answerTime,unixTime,category,created_at,year
0,B00004U9JP,I have a 9 year old Badger 1 that needs replac...,I replaced my old one with this without a hitch.,yes/no,?,"Jun 27, 2014",1.403852e+09,Appliances,2014-06-27 07:00:00,2014.0
1,B00004U9JP,model number,This may help InSinkErator Model BADGER-1: Bad...,open-ended,NaN,"Apr 28, 2014",1.398668e+09,Appliances,2014-04-28 07:00:00,2014.0


In [108]:
#Merge
# Step 1: Add source columns to identify data origin
reddit_final['source'] = 'reddit'
amazon_filt['source'] = 'amazon'

# Step 2: Align column names for merging
reddit_final_renamed = reddit_final.rename(columns={
    'title': 'text',
    'subreddit':'header',
    'created_at': 'timestamp',
    'flair': 'category',
    'year':'year'
})
amazon_filt_renamed = amazon_filt.rename(columns={
    'answer': 'text',
    'question': 'header',
    'created_at': 'timestamp',
    'category': 'category',
    'year': 'year'
})

# Reorder columns for consistency
common_cols = ['header', 'text', 'timestamp', 'category', 'year', 'source']
reddit_final_renamed = reddit_final_renamed[common_cols]
amazon_filt_renamed = amazon_filt_renamed[common_cols]

# Step 3: Merge the datasets
merged_df = pd.concat([reddit_final_renamed, amazon_filt_renamed], ignore_index=True)

# Step 4: Check result
print(merged_df.head())


/tmp/ipykernel_34938/676027832.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_final['source'] = 'reddit'
/tmp/ipykernel_34938/676027832.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_filt['source'] = 'amazon'


         header                                               text  \
0  BuyItForLife  Me with the Golden Taco the day I bought it vs...   
1  BuyItForLife  This 1980's marker, still better than any mode...   
2  BuyItForLife  I’ve wanted one forever and finally found one ...   
3  BuyItForLife                     Best Vacuum for Pet Hair (IMO)   
4  BuyItForLife        NYT just put out a list of their BIFL picks   

            timestamp        category    year  source  
0 2025-04-07 14:00:09         Vintage  2025.0  reddit  
1 2025-04-12 21:25:32         Vintage  2025.0  reddit  
2 2025-03-26 01:54:54  Currently sold  2025.0  reddit  
3 2025-03-25 18:30:44          Review  2025.0  reddit  
4 2025-04-05 20:10:16          Review  2025.0  reddit  


# Sentiment Analysis

In [109]:
!pip install transformers

In [110]:
#pip install pip install numpy==1.24.4

In [111]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, Tokenizer, StopWordsRemover, SQLTransformer
from pyspark.ml.clustering import KMeans
from pyspark.sql.types import FloatType
from sklearn.metrics import accuracy_score, f1_score
from pyspark.sql.types import FloatType, IntegerType
from pyspark.sql.functions import udf, col, when
from transformers import pipeline
import numpy as np

In [112]:
import pandas as pd
import re

def clean_for_sentiment(text):
    if pd.isnull(text):
        return ""

    # Remove line breaks & extra spaces
    text = text.replace("\n", " ").replace("\r", " ")
    
    # Optionally remove URLs (only if model struggles)
    text = re.sub(r'https?://\S+', '', text)
    
    # Trim whitespace
    text = text.strip()

    return text

# Apply to your DataFrame
merged_df['cleaned_text']= merged_df['text'].apply(clean_for_sentiment)
merged_df['cleaned_header']= merged_df['header'].apply(clean_for_sentiment)

In [113]:
!pip install huggingface_hub[hf_xet]


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import pandas as pd

# Load the model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create pipeline (device=-1 = CPU; device=0 = GPU)
classifier = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=-1,                # Change to 0 if you want to use GPU
    truncation=True,
    max_length=512
)

# Correct label map from model config (these are the actual sentiment labels used by this model)
id2label = model.config.id2label  # {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
label_map = {
    "LABEL_0": -1,
    "LABEL_1": 0,
    "LABEL_2": 1
}

# Function to predict sentiment in batches
def predict_bert_labels_batch(texts, batch_size=32):
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        try:
            preds = classifier(batch)
            results.extend([label_map[r["label"]] for r in preds])
        except Exception as e:
            print(f"Error during sentiment prediction on batch {i}-{i+batch_size}: {e}")
            results.extend([None] * len(batch))
    return results

# Example: Assuming you already have a DataFrame named `df`
# And you want to analyze sentiment on a column called 'cleaned_text'

# Step 1: Truncate text manually (optional but safe)
merged_df['cleaned_text'] = merged_df['cleaned_text'].fillna("").apply(lambda x: x[:512])

# Step 2: Run sentiment analysis
merged_df['sentiment_score'] = predict_bert_labels_batch(merged_df['cleaned_text'].tolist())

# Done: Now df has a column with -1 (neg), 0 (neutral), 1 (pos)
print(merged_df[['cleaned_text', 'sentiment_score']].head())


Device set to use cpu


In [ ]:
merged_df.sentiment_score.unique()

In [ ]:
merged_df

# Keyword Extraction

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
from transformers import pipeline
from tqdm.notebook import tqdm
import pandas as pd
import spacy
from collections import Counter

# Load spaCy and NER
nlp = spacy.load("en_core_web_sm")
ner = pipeline("ner", model="Jean-Baptiste/roberta-large-ner-english",
               tokenizer="Jean-Baptiste/roberta-large-ner-english",
               aggregation_strategy="simple", device=0)

# Sample data
#merged_df = qa_df.sample(5000, random_state=42)
merged_df["qa_text"] = merged_df["cleaned_header"].fillna("") + " " + merged_df["cleaned_text"].fillna("")
texts = merged_df["qa_text"].astype(str).tolist()

# Run NER
batch_size = 32
ner_entities = []
labels_to_keep = {"PRODUCT", "ORG", "MISC"}

for i in tqdm(range(0, len(texts), batch_size), desc="Running NER"):
    batch = texts[i:i+batch_size]
    try:
        results = ner(batch)
        batch_words = [
            [ent["word"].lower() for ent in post if ent["entity_group"].upper() in labels_to_keep]
            for post in results
        ]
        ner_entities.extend(batch_words)
    except Exception as e:
        print(f"Batch {i} error:", e)
        ner_entities.extend([[] for _ in batch])

merged_df["ner_entities"] = ner_entities

# Expand entities with noun chunks
def expand_with_noun_chunk(text, ner_words):
    doc = nlp(text)
    expanded = []
    for chunk in doc.noun_chunks:
        for word in ner_words:
            if word in chunk.text.lower():
                expanded.append(chunk.text.lower().strip())
                break
    return list(set(expanded))

merged_df["expanded_entities"] = merged_df.apply(
    lambda row: expand_with_noun_chunk(row["qa_text"], row["ner_entities"]),
    axis=1
)


In [ ]:

# Flatten and attach category
exploded = merged_df[["expanded_entities", "category"]].explode("expanded_entities").dropna()

# Smart phrase cleaning — keep full phrases, remove only unhelpful leading words
def smart_clean(phrase):
    words = phrase.lower().strip().split()
    if len(words) > 1 and words[0] in {"the", "my", "a", "your", "this", "an"}:
        return " ".join(words[1:])
    return " ".join(words)

exploded["clean_phrase"] = exploded["expanded_entities"].apply(smart_clean)

# Group & count
grouped = (
    exploded.groupby(["clean_phrase", "category"])
            .size()
            .reset_index(name="Frequency")
            .sort_values("Frequency", ascending=False)
)

# Show result
print(grouped.head(20))

In [ ]:
# Step 1: Explode with category and date preserved
exploded = merged_df[["expanded_entities", "category", "timestamp","sentiment_score"]].explode("expanded_entities").dropna()

# Step 2: Clean each expanded entity
def smart_clean(phrase):
    words = phrase.lower().strip().split()
    if len(words) > 1 and words[0] in {"the", "my", "a", "your", "this", "an"}:
        return " ".join(words[1:])
    return " ".join(words)

exploded["clean_phrase"] = exploded["expanded_entities"].apply(smart_clean)

# Step 3: Calculate phrase frequency (phrase + category only)
phrase_counts = (
    exploded.groupby(["clean_phrase", "category"])
            .size()
            .reset_index(name="phrase_freq")
)

# Step 4: Join phrase frequency back to the exploded data (not grouped yet)
exploded = exploded.merge(phrase_counts, on=["clean_phrase", "category"], how="left")

# Step 5: Merge back to original `merged_df`
# First, create a unique row ID to merge on
merged_df["row_id"] = merged_df.index
exploded["row_id"] = exploded.index  # exploded keeps same index per row if exploded properly

# Step 6: Group exploded back into list per row_id
final_freq_df = exploded.groupby("row_id")[["clean_phrase", "phrase_freq"]].agg(list).reset_index()

# Step 7: Merge back into merged_df
merged_df = merged_df.merge(final_freq_df, on="row_id", how="left")

# Now merged_df has:
# - original text
# - expanded_entities
# - clean_phrase list
# - phrase_freq list per row


In [ ]:
# Step 1: Explode expanded_entities to one row per entity, keeping metadata
exploded = merged_df[["expanded_entities", "category", "timestamp","sentiment_score"]].copy()
exploded["row_id"] = exploded.index  # Keep reference to original row
exploded = exploded.explode("expanded_entities").dropna()

# Step 2: Smart clean the phrase
#def smart_clean(phrase):
#    words = phrase.lower().strip().split()
#    if len(words) > 1 and words[0] in {"the", "my", "a", "your", "this", "an","any","is","specifially",")","("}:
#        return " ".join(words[1:])
#    return " ".join(words)
leading_trailing_stopwords = {
    "the", "my", "a", "your", "this", "an", "any", "is", "specifially",
    ")", "(", "that", "to", "for", "of", "on", "in", "it","home","page","these","another","not the"
}

def smart_clean(phrase):
    words = phrase.lower().strip().split()

    # Remove leading junk
    while len(words) > 1 and words[0] in leading_trailing_stopwords:
        words = words[1:]

    # Remove trailing junk
    while len(words) > 1 and words[-1] in leading_trailing_stopwords:
        words = words[:-1]

    return " ".join(words)


exploded["clean_phrase"] = exploded["expanded_entities"].apply(smart_clean)

# Step 3: Calculate frequency of each (clean_phrase, category)
phrase_freq = (
    exploded.groupby(["clean_phrase", "category"])
            .size()
            .reset_index(name="phrase_freq")
)

# Step 4: Merge frequency into exploded data
exploded_final = exploded.merge(phrase_freq, on=["clean_phrase", "category"], how="left")

# Step 5: Optional — merge back to original metadata (if needed)
# merged_df can still keep original columns; exploded_final has 1 row per phrase

# Final output: exploded_final
# Columns: [row_id, date, category, expanded_entities, clean_phrase, phrase_freq]


In [ ]:
exploded_final.category.unique()

# TFIDF

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
# Extra stop nouns to remove product-irrelevant words
common_stop_nouns = {
    "what", "anyone", "you", "people", "help","what",'whats',"who","when","why","where", "something", "thing", "someone", "everything",
    "nothing", "everything", "anything", "everything", "this", "that", "one", "some", "time","trump","you"
}
def preprocess_text(text, is_amazon=True, remove_stopwords=False):
    if not isinstance(text, str):
        return ''

    text = text.lower()
    text = re.sub(r'[^a-z0-9\s-]', '', text)

    if not is_amazon:
        text = re.sub(r'@[a-zA-Z0-9_]+', '', text)
        text = re.sub(r'[:;][-o]*[\)p\(d]', '', text)

    tokens = word_tokenize(text)
    all_stopwords = stop_words.union(common_stop_nouns)

    if remove_stopwords:
        print("Before filtering:", tokens)
        tokens = [token for token in tokens if token not in all_stopwords]
        print("After filtering:", tokens)

    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    cleaned_text = ' '.join(tokens).strip()
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text


In [ ]:
exploded_final['cleaned_title_tf'] = exploded_final['clean_phrase'].apply(lambda x: preprocess_text(x, is_amazon=False, remove_stopwords=True))

- Preserves product terms like "air fryer" as "air_fryer" for consistent tokenization 
- Create three clean, tokenized text columns:
    - Column Name	Description
    - TFIDF_Nouns	: Product mentions, entities, core ideas
    - TFIDF_Adjectives	: Opinions, qualities
    - TFIDF_Verbs	: Actions, complaints, behaviors
- Run TF-IDF on each POS slice:
    This gives you POS-specific vocabularies, so you can:
    - POS	Insights
    - **Nouns**	What products/features are mentioned most uniquely?
    - **Adjectives**	What qualities or descriptors are people using?
    - **Verbs**	What do users say they're doing (e.g., “returned”, “replaced”, “loved")


!python -m spacy download en_core_web_sm

### Extract POS (nouns, adjectives, verbs)

In [46]:
!pip install spacy

In [47]:
!python -m spacy download en_core_web_sm


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/runpy.py", line 146, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_pos(text):
    if not isinstance(text, str):
        return [], [], []
    doc = nlp(text)
    nouns = [chunk.text.strip() for chunk in doc.noun_chunks]
    adjectives = [token.text for token in doc if token.pos_ == "ADJ"]
    verbs = [token.text for token in doc if token.pos_ == "VERB"]
    return nouns, adjectives, verbs

# Apply POS tagging to cleaned Reddit titles
pos_data = exploded_final['cleaned_title_tf'].apply(extract_pos)
exploded_final['Cleaned Nouns'] = pos_data.apply(lambda x: x[0])
exploded_final['Cleaned Adjectives'] = pos_data.apply(lambda x: x[1])
exploded_final['Cleaned Verbs'] = pos_data.apply(lambda x: x[2])


### Create TF-IDF–ready text strings

In [ ]:
exploded_final['TFIDF_Nouns'] = exploded_final['Cleaned Nouns'].apply(
    lambda tokens: " ".join(token.replace(" ", "_") for token in tokens)
)

exploded_final['TFIDF_Adjectives'] = exploded_final['Cleaned Adjectives'].apply(
    lambda tokens: " ".join(token.replace(" ", "_") for token in tokens)
)

exploded_final['TFIDF_Verbs'] = exploded_final['Cleaned Verbs'].apply(
    lambda tokens: " ".join(token.replace(" ", "_") for token in tokens)
)

### Run TF-IDF vectorizers

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Nouns
noun_vectorizer = TfidfVectorizer()
X_nouns = noun_vectorizer.fit_transform(exploded_final['TFIDF_Nouns'])

# Adjectives
adj_vectorizer = TfidfVectorizer()
X_adjs = adj_vectorizer.fit_transform(exploded_final['TFIDF_Adjectives'])

# Verbs
verb_vectorizer = TfidfVectorizer()
X_verbs = verb_vectorizer.fit_transform(exploded_final['TFIDF_Verbs'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_top_terms(vectorizer, matrix, top_n=10):
    terms = vectorizer.get_feature_names_out()
    sums = matrix.sum(axis=0).A1
    top_indices = np.argsort(sums)[::-1][:top_n]
    return [(terms[i], round(sums[i], 2)) for i in top_indices]

top_nouns = get_top_terms(noun_vectorizer, X_nouns)
top_adjs  = get_top_terms(adj_vectorizer, X_adjs)
top_verbs = get_top_terms(verb_vectorizer, X_verbs)

def plot_top_terms(top_terms, title):
    terms, scores = zip(*top_terms)
    plt.figure(figsize=(10, 4))
    plt.barh(terms[::-1], scores[::-1])
    plt.title(title)
    plt.xlabel("TF-IDF Score")
    plt.tight_layout()
    plt.show()

plot_top_terms(top_nouns, "Top Product Mentions (Nouns)")
plot_top_terms(top_adjs, "Top Opinion Words (Adjectives)")
plot_top_terms(top_verbs, "Top Action Words (Verbs)")


# Inventory Dataset

In [52]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("malaiarasugraj/e-commerce-dataset")

print("Path to dataset files:",path)
print(os.listdir(path))

In [ ]:
ecomm_df_2 = pd.read_csv(os.path.join(path, "diversified_ecommerce_dataset.csv"))
ecomm_df_2.head()

In [ ]:
ecomm_agg=ecomm_df_2.groupby(['Product Name','Category'])['Stock Level'].sum().reset_index()

In [ ]:
ecomm_agg

# Fuzzy Matching with Amazon dataset

In [ ]:
!pip install sentence_transformers

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load SBERT
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 1: Prepare inputs
phrases = exploded_final[['clean_phrase', 'category', 'timestamp', 'phrase_freq', 'row_id','TFIDF_Nouns']].dropna()
phrases = exploded_final.drop_duplicates(subset=['clean_phrase'])

# Step 2: Embed Reddit clean phrases + Amazon Product Names
reddit_embeddings = model.encode(phrases['clean_phrase'].tolist(), show_progress_bar=True)
amazon_embeddings = model.encode(ecomm_agg['Product Name'].tolist(), show_progress_bar=True)

# Step 3: Match Reddit → Amazon (Top-1 match)
similarity = cosine_similarity(reddit_embeddings, amazon_embeddings)

matches = []
top_k = 1

for i, sim_row in enumerate(similarity):
    top_idx = np.argsort(sim_row)[::-1][:top_k][0]
    matched_product = ecomm_agg.iloc[top_idx]
    
    matches.append({
        'row_id': phrases.iloc[i]['row_id'],
        'clean_phrase': phrases.iloc[i]['clean_phrase'],
        'reddit_category': phrases.iloc[i]['category'],
        'timestamp': phrases.iloc[i]['timestamp'],
        'phrase_freq': phrases.iloc[i]['phrase_freq'],
        'matched_amazon_product': matched_product['Product Name'],
        'amazon_category': matched_product['Category'],
        'stock_level': matched_product['Stock Level'],
        'similarity': sim_row[top_idx]
    })


# Step 4: Final matched DataFrame
matches_df = pd.DataFrame(matches)


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load SBERT
model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare clean and TFIDF noun phrases
phrases = exploded_final[['clean_phrase', 'TFIDF_Nouns', 'category', 'timestamp', 'phrase_freq', 'row_id','sentiment_score']].dropna()
phrases = phrases.drop_duplicates(subset=['clean_phrase'])

# Initial SBERT embeddings on clean_phrase
reddit_embeddings = model.encode(phrases['clean_phrase'].tolist(), show_progress_bar=True)
amazon_embeddings = model.encode(ecomm_agg['Product Name'].tolist(), show_progress_bar=True)

similarity = cosine_similarity(reddit_embeddings, amazon_embeddings)

matches = []
top_k = 1

for i, sim_row in enumerate(similarity):
    top_idx = np.argsort(sim_row)[::-1][:top_k][0]
    sbert_sim = sim_row[top_idx]
    
    # If similarity is low, re-embed using TFIDF_Nouns instead
    if sbert_sim < 0.5:
        tfidf_phrase = phrases.iloc[i]['TFIDF_Nouns']
        tfidf_embedding = model.encode([tfidf_phrase])[0]
        sim_row_alt = cosine_similarity([tfidf_embedding], amazon_embeddings)[0]
        top_idx = np.argsort(sim_row_alt)[::-1][0]
        sim_used = sim_row_alt[top_idx]
    else:
        sim_used = sbert_sim

    matched_product = ecomm_agg.iloc[top_idx]

    matches.append({
        'row_id': phrases.iloc[i]['row_id'],
        'clean_phrase': phrases.iloc[i]['clean_phrase'],
        'category': phrases.iloc[i]['category'],
        'timestamp': phrases.iloc[i]['timestamp'],
        'phrase_freq': phrases.iloc[i]['phrase_freq'],
        'sentiment_score': phrases.iloc[i]['sentiment_score'],
        'matched_product': matched_product['Product Name'],
        'listing_category': matched_product['Category'],
        'stock_level': matched_product['Stock Level'],
        'final_similarity': sim_used,
        'used_backup': sbert_sim < 0.5  # True if TFIDF_Nouns was used
    })

# Final match dataframe
matches_df = pd.DataFrame(matches)


In [ ]:
matched_filt=matches_df[matches_df['final_similarity']>0.5]

In [ ]:
matches_df.to_csv('matched_df_final.csv')

In [ ]:
matched_filt.to_csv('matched_df_final_filt.csv')

In [ ]:
matched_filt['timestamp'].min()

# Streamlit dashboard

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[19]:
import streamlit as st
import pandas as pd
import altair as alt

# -----------------------
# PAGE CONFIG
# -----------------------
st.set_page_config(page_title="TrendNav AI", layout="wide", page_icon="📊")
# -----------------------
# CUSTOM CSS STYLES (UPDATED)
# -----------------------
st.markdown("""
    <style>
    /* Light background for the full app */
    .stApp {
        background-color: #FBFEFF;
    }

    /* Customize the dropdown and selected tag chips */
    div[data-baseweb="select"] > div {
        background-color: #e6f0fa;
        border: 1px solid #0b6da4;
        border-radius: 8px;
    }

    /* Multiselect selected tag styles */
    span[data-baseweb="tag"] {
        background-color: #0b6da4 !important;
        color: white !important;
        border-radius: 5px;
        padding: 5px 8px;
        font-weight: 500;
    }

    /* Column header styling for all tables */
    thead tr th {
        background-color: #0b6da4 !important;
        color: white !important;
        font-weight: bold !important;
        text-align: center;
    }

    /* Table cell alignment */
    tbody td {
        text-align: center;
    }

    /* Remove spacing from top */
    .block-container {
        padding-top: 2rem;
    }
    </style>
""", unsafe_allow_html=True)


# -----------------------
# THEME COLORS
# -----------------------
CARD_COLOR = "#d6f5f2"
TEXT_COLOR = "#0b6da4"
SUMMARY_COLOR = "#444"

# -----------------------
# LOAD DATA
# -----------------------
@st.cache_data
def load_data():
    df = pd.read_csv("matched_df_final_filt.csv", parse_dates=['timestamp'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df['sentiment_score'] = pd.to_numeric(df['sentiment_score'], errors='coerce')
    df['phrase_freq'] = pd.to_numeric(df['phrase_freq'], errors='coerce')
    df['trend_score'] = df['sentiment_score'].fillna(0) + df['phrase_freq'].fillna(0)
    df['sentiment_label'] = df['sentiment_score'].apply(lambda x: 'Positive' if x == 1 else 'Negative' if x == -1 else 'Neutral')
    df['date'] = df['timestamp'].dt.date
    df['month'] = df['timestamp'].dt.to_period('M').astype(str)
    df['week'] = df['timestamp'].dt.strftime('%Y-%U')
    return df

df = load_data()

# -----------------------
# HEADER
# -----------------------
st.markdown(
    f"""
    <h1 style='text-align: center; color: {TEXT_COLOR};'>📊 TrendNav AI: E-commerce Opportunity Scanner</h1>
    <p style='text-align: center; color: {SUMMARY_COLOR}; font-size:20px;'>
        Identifying <strong>trending product demands</strong> using Reddit & Amazon QA,
        and mapping them against <strong>inventory signals</strong> to find high-opportunity areas for sellers.
    </p>
    """, unsafe_allow_html=True
)

# -----------------------
# KPI CARDS (HORIZONTAL)
# -----------------------
# KPIs should use unfiltered data
top_phrase_unfiltered = (
    df.groupby(['clean_phrase', 'matched_product'])
    .agg(total_mentions=('phrase_freq', 'sum'), avg_sentiment=('sentiment_score', 'mean'), trend_score=('trend_score', 'sum'))
    .reset_index()
    .sort_values('trend_score', ascending=False)
)
top_row_unfiltered = top_phrase_unfiltered.iloc[0] if not top_phrase_unfiltered.empty else {}

kpi1, kpi2, kpi3 = st.columns(3)

with kpi1:
    keyword = top_row_unfiltered['clean_phrase'] if not top_phrase_unfiltered.empty else 'N/A'
    subcat = top_row_unfiltered['matched_product'] if not top_phrase_unfiltered.empty else ''
    st.markdown(f"""
    <div style="background-color:{CARD_COLOR}; padding:15px; height:140px; border-radius:12px;">
        <h5 style="color:{TEXT_COLOR};">Top Keyword</h5>
        <h3>{keyword} <span style='font-size: 16px; color: #777;'>({subcat})</span></h3>
    </div>
    """, unsafe_allow_html=True)

with kpi2:
    total_mentions = int(top_row_unfiltered['total_mentions']) if not top_phrase_unfiltered.empty else 0
    st.markdown(f"""
    <div style="background-color:{CARD_COLOR}; padding:15px; height:140px; border-radius:12px;">
        <h5 style="color:{TEXT_COLOR};">Total Mentions</h5>
        <h3>{total_mentions}</h3>
    </div>
    """, unsafe_allow_html=True)

with kpi3:
    avg_sentiment = round(top_row_unfiltered['avg_sentiment'], 2) if not top_phrase_unfiltered.empty else 0.0
    st.markdown(f"""
    <div style="background-color:{CARD_COLOR}; padding:15px; height:140px; border-radius:12px;">
        <h5 style="color:{TEXT_COLOR};">Avg Sentiment</h5>
        <h3>{avg_sentiment}</h3>
    </div>
    """, unsafe_allow_html=True)



# -----------------------
# FILTERS
# -----------------------
# one space row to make the space 
st.markdown("<br>", unsafe_allow_html=True)
st.markdown("<br>", unsafe_allow_html=True)
st.markdown("<br>", unsafe_allow_html=True)
st.markdown(
    """
    <p style='text-align:center; font-size:20px; color:#333;'>
    💡 Want to see what people <strong>love</strong> or <strong>complain</strong> about?  
    Use the filters to explore trending products by sentiment.
    </p>
    """,
    unsafe_allow_html=True
)
# 3 filters in a row
col1, col2, col3 = st.columns(3)

with col1:
    sentiment_choice = st.radio("🧠 Sentiment:", ['All', 'Positive', 'Neutral', 'Negative'], horizontal=True, index=0)

with col2:
    time_choice = st.radio("⏱️ Time View:", ['Daily', 'Weekly', 'Monthly'], horizontal=True)

with col3:
    all_subcats = sorted(df['matched_product'].dropna().unique())
    # Multiselect dropdown sorted by trend score
    sorted_subcats = (
        df.groupby('matched_product')['trend_score']
        .sum()
        .sort_values(ascending=False)
        .index.tolist()
    )

    selected_subcats = st.multiselect(
        "🧵 Subcategory:",
        options=sorted_subcats,
        default=sorted_subcats[:5],  # Optional: pre-select top 5
        help="Filter by one or more top-ranked subcategories"
    )

    
# -----------------------
# APPLY FILTERS to create df_filtered
# -----------------------
df_filtered = df.copy()

if sentiment_choice != 'All':
    df_filtered = df_filtered[df_filtered['sentiment_label'] == sentiment_choice]

if selected_subcats:
    df_filtered = df_filtered[df_filtered['matched_product'].isin(selected_subcats)]


# Apply time view
if time_choice == 'Weekly':
    df_filtered['time_unit'] = df_filtered['week']
elif time_choice == 'Monthly':
    df_filtered['time_unit'] = df_filtered['month']
else:
    df_filtered['time_unit'] = df_filtered['date']

# -----------------------
# TIME FILTER
# -----------------------
# time_choice = st.radio("⏱️ Time View:", ['Daily', 'Weekly', 'Monthly'], horizontal=True)
if time_choice == 'Weekly':
    df_filtered['time_unit'] = df_filtered['week']
elif time_choice == 'Monthly':
    df_filtered['time_unit'] = df_filtered['month']
else:
    df_filtered['time_unit'] = df_filtered['date']

# -----------------------
# SUMMARY LINE
# -----------------------
st.markdown("<br>", unsafe_allow_html=True)

summary_map = {
    'Positive': "🌿 Sentiment is strongly positive — these products are resonating well!",
    'Negative': "⚠️ Negative feedback signals product improvement potential.",
    'Neutral': "🟡 Moderate opinions — could go either way!",
    'All': "📊 Viewing combined sentiment — ideal for overall trend monitoring."
}
st.markdown(f"<p style='text-align:center; color:#333; font-size:20px;'>{summary_map.get(sentiment_choice)}</p>", unsafe_allow_html=True)


# -----------------------
# TRENDING TABLE + CHART
# -----------------------
st.markdown("<br>", unsafe_allow_html=True)

left_col, right_col = st.columns([2, 2.2])

with left_col:
    st.markdown("### 🔥 Top Trending Products")
    top_keywords = (
        df_filtered.groupby(['matched_product', 'clean_phrase'])
        .agg(total_mentions=('phrase_freq', 'sum'),
             avg_sentiment=('sentiment_score', 'mean'),
             trend_score=('trend_score', 'sum'))
        .reset_index()
        .rename(columns={
            'matched_product': 'Product Subcategory',
            'clean_phrase': 'Product Keywords',
            'total_mentions': 'Total Mentions',
            'avg_sentiment': 'Avg Sentiment',
            'trend_score': 'Trend Score'
        })
        .sort_values('Trend Score', ascending=False)
        .head(15)
    )
    st.dataframe(top_keywords, use_container_width=True)

with right_col:
    st.markdown("### 📈 Trend Score Over Time")
    df_trend = df_filtered[df_filtered['timestamp'].dt.year < 2025].copy()
    trend_data = (
        df_trend.groupby(['time_unit', 'matched_product'])
        .agg(trend_score=('trend_score', 'sum'))
        .reset_index()
    )
    chart = alt.Chart(trend_data).mark_line().encode(
    x=alt.X('time_unit:T', title='Date', axis=alt.Axis(grid=False)),
    y=alt.Y('trend_score:Q', title='Trend Score', axis=alt.Axis(grid=False)),
    color='matched_product:N',
    tooltip=['time_unit:T', 'matched_product:N', 'trend_score:Q']
).properties(
    width=520,
    height=390,
    background='#FFFFFF'  # ✅ PERFECT MATCH FROM YOUR TABLE
).configure_axis(
    labelColor='#444',
    titleColor='#0b6da4'
).configure_legend(
    labelColor='#333',
    titleColor='#0b6da4'
).configure_view(
    stroke=None
)


    st.altair_chart(chart, use_container_width=True)

# -----------------------
# OPPORTUNITY ANALYSIS
# -----------------------
st.markdown("### 📦 Opportunity Analysis")

# Use unfiltered for stock-opportunity logic
median_stock = df['stock_level'].median()
median_trend = df['trend_score'].median()

opportunity_df = (
    df.groupby('matched_product')
    .agg(avg_stock=('stock_level', 'mean'), trend_score=('trend_score', 'sum'))
    .reset_index()
)

high_opp = opportunity_df[(opportunity_df['avg_stock'] < median_stock) & (opportunity_df['trend_score'] > median_trend)]
low_opp = opportunity_df[(opportunity_df['avg_stock'] > median_stock) & (opportunity_df['trend_score'] < median_trend)]

col_high, col_low = st.columns(2)
with col_high:
    st.markdown("#### 🟢 High Opportunity Products")
    st.caption("These products are trending but have relatively low stock.")
    if not high_opp.empty:
        st.dataframe(high_opp.rename(columns={'matched_product': 'Product Subcategory', 'avg_stock': 'Avg Stock', 'trend_score': 'Trend Score'}))
    else:
        st.info("No high opportunity products found.")

with col_low:
    st.markdown("#### 📉 Declining Trends")
    st.caption("Products with a significant drop in trend score over the last **month**.")

    # Monthly trend
    df['month'] = df['timestamp'].dt.to_period('M').astype(str)
    monthly_trends = (
        df.groupby(['matched_product', 'month'])['trend_score']
        .sum()
        .reset_index()
        .sort_values(['matched_product', 'month'])
    )
    monthly_trends['pct_change'] = monthly_trends.groupby('matched_product')['trend_score'].pct_change()

    recent_month_drop = (
        monthly_trends.groupby('matched_product').tail(1)
        .query('pct_change < -0.2')
        .sort_values('pct_change')
    )

    if not recent_month_drop.empty:
        display_df = recent_month_drop.rename(columns={
            'matched_product': 'Product Subcategory',
            'trend_score': 'Latest Trend Score',
            'pct_change': '% Change'
        })[['Product Subcategory', 'Latest Trend Score', '% Change']]

        def color_decline(val):
            color = 'red' if val < 0 else 'green'
            return f'color: {color}'

        def center_bold_header():
            return [
                {'selector': 'th', 'props': [('text-align', 'center'), ('font-weight', 'bold')]},
                {'selector': 'td', 'props': [('text-align', 'center')]}
            ]

        st.dataframe(
            display_df
            .style
            .set_table_styles(center_bold_header())
            .applymap(color_decline, subset=['% Change'])
            .format({'Latest Trend Score': '{:,.0f}', '% Change': '{:.0%}'}),
            use_container_width=True
        )
    else:
        st.info("No declining trends detected this month.")